In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

import pathlib as pl

from scipy.sparse import csr_matrix

from statannotations.Annotator import Annotator

from tqdm.notebook import tqdm

In [ ]:
def pretty_ax(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(
        axis='both',  
        which='both',      
        bottom=True,     
        top=False,
        left=False,
        labelbottom=True,
        labelleft = True)
    ax.spines["bottom"].set_linewidth(1.5)
    ax.spines["left"].set_linewidth(1.5)

In [ ]:
datadir = pl.Path("/add/path/here")

In [ ]:
counts = pd.read_csv(datadir / "GSE222078_counts.csv.gz")
counts.index = counts["Unnamed: 0"]
counts = counts.drop("Unnamed: 0", axis=1)
counts = counts.T

In [ ]:
meta = pd.DataFrame("s"+counts.index.str.split("_").str[-1].to_numpy(), index=counts.index, columns=["Sample"])

In [ ]:
meta["Patient"] = meta.Sample.replace({"s1": "P1", "s2": "P2", "s3": "P3", 
                                "s4": "P4", "s5": "P5", "s6": "P6", 
                                "s7": "P7", "s8": "P7", "s9": "P8", 
                                "s10": "P8"})

In [ ]:
meta["Tumor status"] = meta.Sample.replace({"s1": "Tumor", "s2": "Tumor", "s3": "Tumor", 
                                "s4": "Tumor", "s5": "Tumor", "s6": "Tumor", 
                                "s7": "AN", "s8": "Tumor", "s9": "AN", 
                                "s10": "Tumor"})

In [ ]:
X = csr_matrix(counts.round().values)

In [ ]:
genes = pd.DataFrame(index=counts.columns)

In [ ]:
adata = sc.AnnData(X, obs=meta, var=genes)

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata.obs.total_counts.hist(bins=20)

In [ ]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

In [ ]:
adata.write("/add/path/here/GSE222078_adata.h5ad")

# Download program signatures

In [ ]:
signature_dir = pl.Path("/add/path/here/")

full_sigs = {}
for s in (signature_dir).iterdir():
    sig = s.stem
    full_sigs[sig] = pd.read_csv(s,index_col=0)
    full_sigs[sig] = full_sigs[sig][~full_sigs[sig].index.str.startswith(("MT-","RPS","RPL"))]
    full_sigs[sig] = full_sigs[sig].head(100).index.ravel()

# Visualizing signatures in full cohort

In [ ]:
adata = sc.read_h5ad("/add/path/here/GSE222078_adata.h5ad")

In [ ]:
celltypes = pd.read_csv(datadir / "highLevelCellTypes.csv",index_col=0)

celltypes.index = celltypes.index.str.replace("-",".")

adata.obs = pd.concat([adata.obs,celltypes],axis=1)

In [ ]:
sc.tl.pca(adata)

sc.external.pp.harmony_integrate(adata, key="Sample", max_iter_harmony=20)

sc.pp.neighbors(adata, use_rep="X_pca_harmony")

sc.tl.umap(adata)

In [ ]:
toptfs = pd.read_csv("/add/path/here/toptfs_top20.csv",index_col=0)
toptfs = {state[:6]: toptfs[state].dropna().ravel() for state in toptfs.columns}

In [ ]:
for sig, genes in full_sigs.items():
    sc.tl.score_genes(adata=adata,
                        gene_list=np.setdiff1d(genes,toptfs[sig]), 
                        score_name=f"{sig}_score")

In [ ]:
cnmf_names = [f"cNMF_{i}_score" for i in range(1,6)]

In [ ]:
sc.pl.umap(adata, color=["celltype"]+cnmf_names,ncols=2)

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(10,10))
flatax = axs.flatten()
for i,ax in enumerate(flatax[:-1]):
    sns.boxplot(data=adata.obs, x="celltype", y=f"cNMF_{i+1}_score", ax=flatax[i])
    flatax[i].spines[['right', 'top']].set_visible(False)
    flatax[i].set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    flatax[i].hlines(y=adata.obs[f"cNMF_{i+1}_score"].median(), xmin=flatax[i].get_xlim()[0], xmax=flatax[i].get_xlim()[1], linestyles="dashed", color="grey")
flatax[-1].axis("off")
fig.tight_layout()
fig.savefig("figures/external/boxplot_cNMF_score_per_celltype.png", dpi=250, bbox_inches="tight")

# Visualizing signatures in carcinoma cells

In [ ]:
subadata = adata[adata.obs.celltype.isin(["Epithelial"])].copy()

In [ ]:
sc.tl.pca(subadata)

sc.external.pp.harmony_integrate(subadata, key="Sample", max_iter_harmony=20, sigma=0.6)

sc.pp.neighbors(subadata, use_rep="X_pca_harmony")

sc.tl.umap(subadata)

In [ ]:
sc.pl.umap(subadata, color=cnmf_names,ncols=2, frameon=False)

In [ ]:
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as mcolors

X_pca = pd.DataFrame(subadata.obsm["X_pca_harmony"][:,:2],index=subadata.obs_names,columns=["PC1","PC2"])

X_pca = pd.concat([X_pca, subadata.obs[["cNMF_1_score","cNMF_2_score","cNMF_3_score",
                                        "cNMF_4_score","cNMF_5_score"]]],axis=1)
X_pca.columns = ["PC1","PC2","cNMF_1","cNMF_2","cNMF_3","cNMF_4","cNMF_5"]

def plot_pcs_color(ax, state):
    vcenter = X_pca[state].median()
    vmin, vmax = X_pca[state].min(), X_pca[state].max()
    normalize = mcolors.TwoSlopeNorm(vcenter=vcenter, vmin=vmin, vmax=vmax)
    colormap = matplotlib.colormaps['RdBu_r']
    sns.scatterplot(
        y=X_pca["PC2"],
        x=X_pca["PC1"],
        c=X_pca[state],
        s=15,
        norm=normalize,
        cmap=colormap,
        ax=ax
    )
    scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
    scalarmappaple.set_array(X_pca[state])
    fig.colorbar(scalarmappaple, ax=ax)
    ax.set_title(state)
    pretty_ax(ax)

fig, ax = plt.subplots(1,5, figsize=(15,2))
flatax = ax.flatten()

plot_pcs_color(flatax[0], "cNMF_1")
plot_pcs_color(flatax[1], "cNMF_2")
plot_pcs_color(flatax[2], "cNMF_3")
plot_pcs_color(flatax[3], "cNMF_4")
plot_pcs_color(flatax[4], "cNMF_5")

fig.tight_layout()


In [ ]:
subadata.obs[["cNMF_1_score","cNMF_2_score","cNMF_3_score","cNMF_4_score","cNMF_5_score"]].corr()